# Installing environment

##  Importing modules and libraries

In [8]:
from IPython.display import HTML # to display videos
import base64 # to encode videos as base64
from base64 import b64encode # to encode videos as base64
import os # to interact with the operating system
import subprocess # to run commands
import time # to measure execution time
import csv # to save comments
import uuid # to generate unique ids
import cv2 # to split videos
from PIL import Image # to display videos
import pandas as pd # to display comments
import numpy as np # to use Numerical Python
from io import BytesIO #to manage a binary stream of data in memory

In [9]:
def download(directory, filename):
    # The base URL of the image files in the GitHub repository
    base_url = 'https://raw.githubusercontent.com/Denis2054/RAG-Driven-Generative-AI/main/'

    # Complete URL for the file
    file_url = f"{base_url}{directory}/{filename}"

    # Use curl to download the file, including an Authorization header for the private token
    try:
        # Prepare the curl command with the Authorization header
        curl_command = f'curl -o {filename} {file_url}'

        # Execute the curl command
        subprocess.run(curl_command, check=True, shell=True)
        print(f"Downloaded '{filename}' successfully.")
    except subprocess.CalledProcessError:
        print(f"Failed to download '{filename}'. Check the URL, your internet connection and the file path")

## OpenAI

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
f = open("drive/MyDrive/api_key.txt", "r")
API_KEY=f.readline()
f.close()

os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

In [12]:
try:
  import openai
except:
  !pip install openai==1.45.0
import openai

## Pinecone

In [13]:
!pip install pinecone
import pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 22.3 MB/s eta 0:00:00


In [14]:
f = open("drive/MyDrive/pinecone.txt", "r")
PINECONE_API_KEY=f.readline()
f.close()

# Displaying the video

In [15]:
# Open the file in binary mode
def display_video(file_name):
  with open(file_name, 'rb') as file:
      video_data = file.read()

  # Encode the video file as base64
  video_url = b64encode(video_data).decode()

  # Create an HTML string with the embeddZed video
  html = f'''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{video_url}" type="video/mp4">
  Your browser does not support the video tag.
  </video>
  '''
  # Display the video
  HTML(html)
  # Return the HTML object
  return HTML(html)

# Splitting video into frames

In [16]:
def split_file(file_name):
  video_path = file_name
  cap = cv2.VideoCapture(video_path)

  frame_number = 0
  while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
      break
    cv2.imwrite(f"frame_{frame_number}.jpg", frame)
    frame_number += 1
    print(f"Frame {frame_number} saved.")
  cap.release()
# Generator job's finished here.

# Commenting on the frames

In [17]:
# Commentator's extraction fn
def generate_comment(response_data):
  """Extract relevant information from GPT-4 Vision response."""
  try:
    caption = response_data.choices[0].message.content
    return caption
  except (KeyError, AttributeError):
    print("Error extracting caption from response.")
    return "No caption available"

In [18]:
def save_comment(comment, frame_number, file_name):
  """Save the comment to a text file formatted for seamless loading into a pandas DF."""
  path = f"{file_name}.csv"
  write_header = not os.path.exists(path)

  with open(path, 'a', newline='') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    if write_header:
      writer.writerow(['ID', 'FrameNumber', 'Comment', 'FileName']) # Write the header if the file is being created
    unique_id = str(uuid.uuid4())
    writer.writerow([unique_id, frame_number, comment, file_name])

In [19]:
import base64
import requests

def generate_openai_comments(filename):
  video_folder = "/content"  # Folder containing your image frames
  total_frames = len([file for file in os.listdir(video_folder) if file.endswith('.jpg')])

  nb=3      # sample frequency
  counter=0 #sample frequency counter
  for frame_number in range(total_frames):
      counter+=1 # sampler
      if counter==nb and counter<total_frames:
        counter=0
        print(f"Analyzing frame {frame_number}...")
        image_path = os.path.join(video_folder, f"frame_{frame_number}.jpg")
        try:
            with open(image_path, "rb") as image_file:
                base64_image=base64.b64encode(image_file.read()).decode('utf-8')
            #with open(image_path, "rb") as image_file:
                #image_data = image_file.read()

                response = openai.chat.completions.create(
                    model="gpt-4o",
                    messages=[
                        {
                            "role": "user",
                            "content": [
                                {"type": "text", "text": "What is happening in this image?"},
                                {
                                    "type": "image_url",
                                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
                                    #"image_url": f"data:image/jpeg;base64,{base64.b64encode(image_data).decode('utf-8')}"
                                },
                            ],
                       }
                    ],
                    max_tokens=150,
               )
            comment = generate_comment(response)
            save_comment(comment, frame_number,file_name)

        except FileNotFoundError:
            print(f"Error: Frame {frame_number} not found.")
        except Exception as e:
            print(f"Unexpected error: {e}")

In [23]:
def display_comments(file_name):
  path = f"{file_name}.csv"
  df = pd.read_csv(path)
  return df

# Pipeline 1 controller

The controller runs jobs for the preceding three steps of the Generator and the Commentator.

In [21]:
session_time = time.time()

# Displaying the video
print("Step 1: Collecting video")
file_name = "skiing1.mp4"
print(f"Video: {file_name}")

# Downloading the video
print("Step 1: downloading from Github")
directory = "Chapter10/videos"
download(directory, file_name)

# Displaying video
print("Step 1: displaying video")
display_video(file_name)

# Splitting video
print("Step 2: splitting the video into frames")
split_file(file_name)

# Commenting on the video frames
print("Step 3: commenting on the frames")
start_time = time.time()
generate_openai_comments(file_name)
response_time = time.time() - session_time

Step 1: Collecting video
Video: skiing1.mp4
Step 1: downloading from Github
Downloaded 'skiing1.mp4' successfully.
Step 1: displaying video
Step 2: splitting the video into frames
Frame 1 saved.
Frame 2 saved.
Frame 3 saved.
Frame 4 saved.
Frame 5 saved.
Frame 6 saved.
Frame 7 saved.
Frame 8 saved.
Frame 9 saved.
Frame 10 saved.
Frame 11 saved.
Frame 12 saved.
Frame 13 saved.
Frame 14 saved.
Frame 15 saved.
Frame 16 saved.
Frame 17 saved.
Frame 18 saved.
Frame 19 saved.
Frame 20 saved.
Frame 21 saved.
Frame 22 saved.
Frame 23 saved.
Frame 24 saved.
Frame 25 saved.
Frame 26 saved.
Frame 27 saved.
Frame 28 saved.
Frame 29 saved.
Frame 30 saved.
Frame 31 saved.
Frame 32 saved.
Frame 33 saved.
Frame 34 saved.
Frame 35 saved.
Frame 36 saved.
Frame 37 saved.
Frame 38 saved.
Frame 39 saved.
Frame 40 saved.
Frame 41 saved.
Frame 42 saved.
Frame 43 saved.
Frame 44 saved.
Frame 45 saved.
Frame 46 saved.
Frame 47 saved.
Frame 48 saved.
Frame 49 saved.
Frame 50 saved.
Frame 51 saved.
Frame 52 save

In [24]:
video_folder = "/content"
total_frames = len([file for file in os.listdir(video_folder) if file.endswith(".jpg")])
print(total_frames)

print("Commenting video: displaying comments.")
display_comments(file_name)

total_time = time.time() - start_time
print(f"Response time: {response_time:.2f} seconds")
print(f"Total time: {total_time:.2f} seconds")

57
Commenting video: displaying comments.
Response time: 70.70 seconds
Total time: 205.44 seconds


# Saving comments and frames

In [27]:
save=False
save_frames=False

if save:
  cpath = f"{file_name}.csv"
  if os.path.exists(cpath):
    !cp {path} /content/drive/MyDrive/comments/{cpath}
    print(f"File {cpath} copied successfully.")
  else:
    print(f"No such file: {cpath}")

In [28]:
# Save frames
import shutil

if save_frames:
  # Extract the root name by removing the extension
  root_name, extension = os.path.splitext(file_name)

  # This removes the period from the extension
  root_name = root_name + extension.strip('.')

  # path where you want to copy the jpg files
  target_directory = f'/content/drive/MyDrive/comments/{root_name}'

  # Ensure the directory exists
  os.makedirs(target_directory, exist_ok=True)

  # Assume your jpg files are in the current directory
  source_directory = os.getcwd()

  # list files in the source dir
  for file in os.listdir(source_directory):
    if file.endswith('.jpg'):
      shutil.copy(os.path.join(source_directory, file), target_directory)

# Deleting files

In [29]:
delf=False     # double checking before deleting the files in a session
if delf==True:
  !rm -f *.mp4 # video files
  !rm -f *.jpg # frames
  !rm -f *.csv # comments